In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [25]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [26]:
design.chips['C_chip'].size['size_x']='3mm'
design.chips['C_chip'].size['size_y']='3mm'
design.chips['Q_chip'].size['size_x']='2mm'
design.chips['Q_chip'].size['size_y']='2mm'
design.chips['Q_chip'].size['center_z']='7um'

In [27]:
design.get_chip_z('Q_chip')

'7um'

In [40]:

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-300um',
                                            pos_y='-500um',
                                            chip='Q_chip',
                                            pad_width='390um',
                                            pad_height='130um',
                                            pad_gap='60um',
                                            pocket_width='680um',
                                            pocket_height='680um'))]
                                            
qubits=q0
q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-120}um",
                                              pos_y=f"{q0y+90}um",
                                              pad_width="165um",
                                              pad_height="130um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))
design.rebuild()
gui.rebuild()
gui.autoscale()

In [30]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-700um',
                                         pos_y = '800um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '700um',
                                         pos_y = '800um',
                                         orientation='180',
                                         lead_length = '30um'))                                        

In [31]:
stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='0.65mm',
                                                pos_y='-0.2mm',
                                                orientation='270'))

In [32]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [33]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Resonator length
3.85mm

In [34]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [35]:
resonator_length_4=[]
a=find_resonator_length(7.5,10,6,4)
a=a.strip('mm')
a=float(a)-0.25 +0.5-0.23-0.5# subtract coupler length, for +0.5Ghz, coupler length
resonator_length_4.append(a)
print(resonator_length_4)


[3.5775622473488387]


In [36]:
jogs1=OrderedDict()
jogs1[0]=['R','300um']
jogs1[1]=['R', '300um']
jogs1[2]=['L','700um']
jogs1[3]=['R','200um']
jogs1[4]=['R','670um']
jogs1[5]=['L','200um']
jogs1[6]=['L','740um']


In [44]:
rr0=connect('rr0', 'c0', 'in','stg', 'short',
            resonator_length_4[0],jogs1,jogs, 
            '100um','0um', '929um', '100um')
            

# EPR Analysis

In [45]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [46]:
hfss.start()

INFO 06:14PM [connect_project]: Connecting to Ansys Desktop API...
INFO 06:14PM [load_ansys_project]: 	Opened Ansys App
INFO 06:14PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 06:14PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   bump_V7
INFO 06:14PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 06:14PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 06:14PM [connect]: 	Connected to project "bump_V7" and design "Design_q3d" 😀 



True

In [47]:
hfss.activate_ansys_design("flip7um_V8", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

06:14PM 42s WARNING [activate_ansys_design]: The design_name=flip7um_V8 was not in active project.  Designs in active project are: 
['Design_q3d'].  A new design will be added to the project.  
INFO 06:14PM [connect_design]: 	Opened active design
	Design:    flip7um_V8 [Solution type: Eigenmode]
WARNING 06:14PM [connect_setup]: 	No design setup detected.
WARNING 06:14PM [connect_setup]: 	Creating eigenmode default setup.
INFO 06:14PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [20]:
hfss.close()


True

# LOM Analysis

In [19]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [20]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.82',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.82',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [22]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 06:08PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp_x_g1lm4.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5820000000, Maxwell, 1, False
INFO 06:08PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpz6nosmri.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5820000000, Maxwell, 1, False
INFO 06:08PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpkgvlvlj_.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5820000000, Maxwell, 2, False
INFO 06:08PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp4d4p47ln.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5820000000, Maxwell, 3, False
INFO 06:08PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpeo85y7vi.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5820000000, Maxwell, 4, False
INFO 06:08

,pad_bot_Q0,pad_top_Q0,quad_coupler_c0,SignalNet1
pad_bot_Q0,130.10000,-11.920,-0.15053,-116.570
pad_top_Q0,-11.92000,130.080,-22.59400,-93.914
quad_coupler_c0,-0.15053,-22.594,113.92000,-91.030
SignalNet1,-116.57000,-93.914,-91.03000,365.110


In [23]:
c1.setup.junctions = Dict({'Lj': 12, 'Cj': 2})
c1.setup.freq_readout = 7.02
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.078024 [GHz]
EC 263.271301 [MHz]
EJ 13.616300 [GHz]
alpha -301.495309 [MHz]
dispersion 7.705657 [KHz]
Lq 11.995161 [nH]
Cq 73.575160 [fF]
T1 218.293455 [us]

**Coupling Properties**

tCqbus1 -5.435846 [fF]
gbus1_in_MHz -74.551200 [MHz]
χ_bus1 -0.792712 [MHz]
1/T1bus1 729.087106 [Hz]
T1bus1 218.293455 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,6.288821,416.429131,13.6163,-500.370906,422.388532,[-52.95497849019796],[-3.132911137943207],3.132911,52.954978
2,5.126435,268.632114,13.6163,-308.168528,9.384361,[-50.962023653855354],[-0.39507488862295537],0.395075,50.962024
3,5.151226,271.401187,13.6163,-311.623417,10.365869,[-70.37017682861419],[-0.778815833037783],0.778816,70.370177
4,5.116536,267.530971,13.6163,-306.796163,9.016468,[-73.90054129688407],[-0.8198426810518622],0.819843,73.900541
5,5.099673,265.661086,13.6163,-304.467656,8.419609,[-74.84798226334928],[-0.8223075311348527],0.822308,74.847982
6,5.094135,265.04851,13.6163,-303.70537,8.231484,[-73.83786173053848],[-0.7943934075605666],0.794393,73.837862
7,5.078024,263.271301,13.6163,-301.495309,7.705657,[-74.5512003200581],[-0.7927117121282262],0.792712,74.551200


In [24]:
c1.sim.close()

Warning! 6 COM references still alive
Ansys will likely refuse to shut down
